#### <font color=blue>BaselineOnly</font>

[UCSD](https://cseweb.ucsd.edu/~jmcauley/datasets.html#clothing_fit) dataset on clothing fit

**Item-based CF**:
- <font color=red>items usually don't change much</font> so this approach can be computed offline
- uses patterns of users who browsed the same item as active user
- <font color=blue>"Users who liked this item also liked ..."</font>

**User-based CF**:
- use k-nearest neighbors to find <font color=red>clusters of similar users based on common item ratings</font>
- then make predictions using the <font color=red>average rating of top k-nearest neighbors</font>

- <font color=blue>"Users who are similar to you also liked ..."</font>

In [1]:
## importing modules 

import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
% config InlineBackend.figure_format = 'retina'
plt.style.use('fivethirtyeight')
import itertools

# surprise modules
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise.model_selection import KFold
from surprise.model_selection import GridSearchCV
from surprise import NormalPredictor
from surprise import BaselineOnly
from surprise import SlopeOne
from surprise import CoClustering
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import SVD
from surprise import SVDpp
from surprise import NMF
from surprise import accuracy
from surprise.accuracy import rmse

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns',None)

In [2]:
## read csv file

df = pd.read_csv('renttherunway_reduced25_new.csv')

print(df.shape)
df.head()

(1874, 26)


,age,body type,bust size,category,fit,height,item_id,rating,rented for,review_date,review_summary,review_text,size,user_id,weight,band_size,cup_size,feet,inches,height_inch,weight_lbs,year,month,day,item_id_size,item_id_new
0,32.0,hourglass,32c,dress,fit,"5' 3""",143094,5.0,party,2015-04-07,glamorous like a sexy disco ball!,few friends and i wore this dress and went to ...,8,610914,140lbs,32.0,c,5,3,63,140.0,2015,4,7,14309408,751.0
1,38.0,straight & narrow,34c,dress,fit,"5' 8""",143094,2.0,vacation,2012-11-25,looked like a showstopper but i was itching al...,"the dress fit great, but the sequins did not l...",8,53519,135lbs,34.0,c,5,8,68,135.0,2012,11,25,14309408,751.0
2,37.0,athletic,36c,dress,fit,"5' 4""",143094,5.0,party,2014-04-15,perfect dress for my bachelorette party!,fit perfectly without being too tight. materia...,8,470639,138lbs,36.0,c,5,4,64,138.0,2014,4,15,14309408,751.0
3,47.0,athletic,34b,dress,fit,"5' 5""",145417,5.0,other,2014-01-15,great for work-to-dinner party,form fitting and flattering. this dress was c...,8,276186,120lbs,34.0,b,5,5,65,120.0,2014,1,15,14541708,804.0
4,36.0,pear,34c,dress,fit,"5' 7""",145417,4.0,party,2016-01-20,charity event,this dress is a great color and is a nubby mat...,8,748397,125lbs,34.0,c,5,7,67,125.0,2016,1,20,14541708,804.0


In [3]:
## Ratings distribution

df['rating'].value_counts()

5.0    1220
4.0     495
3.0     123
2.0      28
1.0       8
Name: rating, dtype: int64

In [4]:
df['user_id'].nunique()

755

In [5]:
df['item_id_new'].nunique()

797

<div class='alert alert-danger'>
#### Using the Surprise module
- load dataset from pandas dataframe using `Dataset.load_from_df`

#### Algorithms to compare:
- `NormalPredictor`: predicts random rating based on training set which is assumed to be normally distributed
- `BaselineOnly`: predicts the baseline estimate for a given user and given item
- `Slope One`: factors in items that a user liked separately from items that a user disliked (by computing the average difference between ratings of one item and another for users who rated both)
- `Co-clustering`: simultaneous clustering of users and items

**Neighborhood collaborative filtering (CF)**:
- `KNNBasic`: basic nearest neighbors CF algorithm
- `KNNWithMeans`: takes into account the mean ratings of each user
- `KNNWithZScore`: takes into account the z-score normalization of each user
- `KNNBaseline`: takes into account a baseline rating

**Matrix factorization CF**:
- `SVD`: singular value decomposition is equivalent to [Probabilistic Matrix Factorization, PMF](http://papers.nips.cc/paper/3208-probabilistic-matrix-factorization.pdf) if baselines are not used
- `SVDpp or SVD++`: takes into account implicit ratings
- `NMF`: non-negative matrix factorization

#### Model selection:
- manually split data into training set and test set (80-20)
- use `cross_validate` to select model with lowest RSME score
- use `GridSearchCV` to tune hyperparameters for ratings prediction

In [6]:
## create reader and load specific columns from dataframe into surprise

reader = Reader(rating_scale=(1,5))

data = Dataset.load_from_df(df[['user_id','item_id_new','rating']], reader)

data1 = Dataset.load_from_df(df[['user_id','item_id_new','rating']], reader)

<div class='alert alert-warning'>
#### [manually split data](https://surprise.readthedocs.io/en/stable/FAQ.html#how-to-save-some-data-for-unbiased-accuracy-estimation) into training set and test set

In [7]:
raw_ratings = data.raw_ratings

random.shuffle(raw_ratings)

# set training set as 80% of the data, test set as 20%
threshold = int(0.8 * len(raw_ratings))

train_raw_ratings = raw_ratings[:threshold]
test_raw_ratings = raw_ratings[threshold:]

data.raw_ratings = train_raw_ratings

<div class='alert alert-warning'>
#### compare algorithms using cross validation and pre-defined training set (80%)
- using metrics: `RMSE`, `MAE`

In [8]:
comparison = []

algos = [NormalPredictor(), BaselineOnly(), SlopeOne(), CoClustering(),\
         KNNBasic(), KNNWithMeans(), KNNWithZScore(), KNNBaseline(),\
         SVD(), SVDpp(), NMF()]

for algorithm in algos:
    score = cross_validate(algorithm, data, measures=['RMSE','MAE'],\
                             cv=5, verbose=False)
    
    result = pd.DataFrame.from_dict(score).mean(axis=0)
    result = result.append(pd.Series([str(algorithm).split(' ')[0].\
                                      split('.')[-1]],\
                                     index=['Algorithm']))
    comparison.append(result)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Don

In [9]:
pd.DataFrame(comparison).set_index('Algorithm').\
sort_values('test_rmse',ascending=True)

,test_rmse,test_mae,fit_time,test_time
Algorithm,,,,
SVDpp,0.718094,0.574995,0.190531,0.004922
BaselineOnly,0.718583,0.585430,0.004629,0.001855
KNNBasic,0.722063,0.593298,0.007969,0.004966
SVD,0.722208,0.581793,0.088207,0.035633
KNNBaseline,0.722897,0.586878,0.012171,0.003039
SlopeOne,0.817845,0.593147,0.019058,0.002861
KNNWithMeans,0.817911,0.593796,0.014404,0.003317
KNNWithZScore,0.830081,0.591335,0.033119,0.003436
CoClustering,0.868189,0.639699,0.138463,0.002229


<div class='alert alert-danger'>
#### use the 4 best algorithms to get predictions for existing testset ratings: 
- `BaselineOnly()`, `SVDpp()`, `SVD()`, `KNNWithZScore()`
- create dataframe for predictions of existing testset ratings
- note: lower RMSE/MAE is better

In [10]:
def get_items(user_id):
    """ returns the number of items rated by a given user
    args: 
      uid: the 'raw' id of the user
    """
    try:
        return len(trainset.ur[trainset.to_inner_uid(user_id)])
            # to_inner_uid() converts raw user id to inner id

    except ValueError: # user was not part of the trainset
        return 0
    
def get_users(item_id):
    """ returns number of users that have rated a given item
    args:
      iid: the raw id of the item
    """
    try: 
        return len(trainset.ir[trainset.to_inner_iid(item_id)])
            # to_inner_iid() converts raw item id to inner id
    except ValueError:
        return 0

### predict using BaselineOnly

In [11]:
## use Alternating Least Squares (ALS) for BaselineOnly

trainset = data.build_full_trainset()

bsl_options = {'method':'als', 'n_epochs': 5, 'reg_u':12, 'reg_i': 5}

algo1 = BaselineOnly(bsl_options=bsl_options)
algo1.fit(trainset)

predictions1_biased = algo1.test(trainset.build_testset())
print('Biased accuracy on training set (BaselineOnly): ', end='   ')
accuracy.rmse(predictions1_biased)

###########################################################
testset = data.construct_testset(test_raw_ratings)
predictions1_unbiased = algo1.test(testset)
print('Unbiased accuracy on test set (BaselineOnly): ', end='   ')
accuracy.rmse(predictions1_unbiased)

Estimating biases using als...
Biased accuracy on training set (BaselineOnly):    RMSE: 0.5878
Unbiased accuracy on test set (BaselineOnly):    RMSE: 0.7266


0.72660380759287

In [12]:
# checking that the train-test split is indeed 80-20

print('length of the train set: ',trainset.n_ratings)
print('length of the test set: ',len(testset))

length of the train set:  1499
length of the test set:  375


In [13]:
## top 10 best and worst predictions for the testset using BaselineOnly

df_BsO = pd.DataFrame(predictions1_unbiased, columns=['uid','iid','actual rating',\
                                                      'estimated rating','details'])
df_BsO['number of items for target user'] = df_BsO['uid'].apply(get_items)
df_BsO['number of users for target item'] = df_BsO['iid'].apply(get_users)
df_BsO['error'] = abs(df_BsO['estimated rating'] - df_BsO['actual rating'])

Best_predictions_BsO = df_BsO.sort_values(by='error')[:10]
Best_predictions_BsO

,uid,iid,actual rating,estimated rating,details,number of items for target user,number of users for target item,error
197,551741,68.0,4.0,3.965975,{'was_impossible': False},1,2,0.034025
338,995412,72.0,4.0,3.934905,{'was_impossible': False},3,5,0.065095
302,439590,72.0,4.0,4.106401,{'was_impossible': False},4,5,0.106401
200,327294,16609.0,4.0,4.109715,{'was_impossible': False},4,1,0.109715
142,290985,18707.0,5.0,4.844951,{'was_impossible': False},6,2,0.155049
52,507493,11966.0,5.0,4.839009,{'was_impossible': False},3,4,0.160991
27,32925,22933.0,5.0,4.829292,{'was_impossible': False},5,3,0.170708
92,393306,72.0,4.0,4.177287,{'was_impossible': False},0,5,0.177287
40,897249,11507.0,5.0,4.808972,{'was_impossible': False},0,9,0.191028
309,890869,27119.0,5.0,4.808891,{'was_impossible': False},0,7,0.191109


In [14]:
Worst_predictions_BsO = df_BsO.sort_values(by='error')[-10:]
Worst_predictions_BsO

,uid,iid,actual rating,estimated rating,details,number of items for target user,number of users for target item,error
243,741924,27529.0,3.0,4.615715,{'was_impossible': False},0,6,1.615715
110,930348,361.0,3.0,4.640789,{'was_impossible': False},1,1,1.640789
325,776382,9608.0,3.0,4.646212,{'was_impossible': False},1,1,1.646212
44,388317,1366.0,3.0,4.772748,{'was_impossible': False},2,3,1.772748
319,778579,25372.0,2.0,4.330153,{'was_impossible': False},4,3,2.330153
260,96369,16162.0,2.0,4.414175,{'was_impossible': False},3,2,2.414175
352,432828,29295.0,2.0,4.549033,{'was_impossible': False},0,0,2.549033
11,645325,169.0,2.0,4.585268,{'was_impossible': False},0,3,2.585268
286,933801,1666.0,1.0,4.574656,{'was_impossible': False},1,0,3.574656
116,497208,12287.0,1.0,4.673311,{'was_impossible': False},2,1,3.673311


<div class='alert alert-warning'>
#### get top-N recommendations for each user

- first train each algorithm on the whole dataset
- then predict all the **ratings for user-item pairs that are not in** the dataset
- retrieve top-N predictions for each user

In [15]:
## using get_top_n function from surprise library

from collections import defaultdict

def get_top_n(predictions, n):
    ''' 
    Return the top-N recommendation for each user from set of predictions.
        
    Args:
    predictions(list of Prediction objects): The list of predictions,
        as returned by the test method of an algorithm.
    n(int): The number of recommendation to output for each user. 
    Default is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of
        tuples: [(raw item id, rating estimation), ...] of size n.
    '''
        
    # First map the predictions to each user.
    top_n = defaultdict(list)
    
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the 
        # k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n


def get_categ(itemID):
    ''' 
    Returns item category for a given itemID
    '''
    category = df[df['item_id_new']==itemID]['category'].values[0]
    
    return category


def get_existing(userID):
    ''' 
    Returns list of at most 10 random items that user has rented
    '''
    existing = df[df['user_id']==userID]['item_id_new']
    exdict = {}
    if len(existing) < 10:
        exist = list(df[df['user_id']==userID]['item_id_new'].\
                     sample(n=len(existing), random_state=1))
        exdict[userID] = [(a, get_categ(a)) for a in exist]
    else:
        exist = list(df[df['user_id']==userID]['item_id_new'].\
                     sample(n=10, random_state=1))
        exdict[userID] = [(a, get_categ(a)) for a in exist]
    return exdict

In [16]:
## train BaselineOnly algorithm on the whole dataset

trainset = data1.build_full_trainset()

bsl_options = {'method':'als', 'n_epochs': 5, 'reg_u':12, 'reg_i': 5}

algo = BaselineOnly(bsl_options=bsl_options)
algo.fit(trainset)

## predict all ratings (for user-item pairs) that are not in the train set
testset = trainset.build_anti_testset()
predictions_BsO = algo.test(testset)
accuracy.rmse(predictions_BsO)

topNPredicted_BsO = get_top_n(predictions_BsO, 10)

BsO_recc_dict = {}
BsO_exist_dict = {}

## print existing and recommended items for each user:
i=0
for uid, user_ratings in topNPredicted_BsO.items():
    exitem = get_existing(uid)
    
    print("Random 10 rented items for user {0}: {1}".\
          format(uid, list(a for a in exitem.values())[0]))
    BsO_exist_dict[uid] = list(a for a in exitem.values())[0]
    print()
    
    BsO_recc_dict[uid] = [(iid,get_categ(iid)) for (iid,_) in user_ratings]
    print('Top 10 recommended new items for user {0}: {1}'.\
          format(uid, [(iid,get_categ(iid)) for (iid,_) in user_ratings]))
    print('-------------------------------------------------------------')
    
    i+=1
print('Number of recommended item sets (BaselineOnly):', i)

Estimating biases using als...
RMSE: 0.1561
Random 10 rented items for user 610914: [(751.0, 'dress')]

Top 10 recommended new items for user 610914: [(27119.0, 'jumpsuit'), (11507.0, 'dress'), (11966.0, 'dress'), (27357.0, 'blazer'), (4816.0, 'sheath'), (544.0, 'sheath'), (17879.0, 'dress'), (528.0, 'dress'), (10883.0, 'sheath'), (2171.0, 'dress')]
-------------------------------------------------------------
Random 10 rented items for user 53519: [(751.0, 'dress'), (8003.0, 'dress')]

Top 10 recommended new items for user 53519: [(27119.0, 'jumpsuit'), (11507.0, 'dress'), (11966.0, 'dress'), (27357.0, 'blazer'), (4816.0, 'sheath'), (544.0, 'sheath'), (17879.0, 'dress'), (528.0, 'dress'), (10883.0, 'sheath'), (2171.0, 'dress')]
-------------------------------------------------------------
Random 10 rented items for user 470639: [(414.0, 'gown'), (528.0, 'dress'), (22588.0, 'dress'), (751.0, 'dress'), (878.0, 'dress')]

Top 10 recommended new items for user 470639: [(27119.0, 'jumpsuit

Random 10 rented items for user 968500: [(29560.0, 'jumpsuit'), (169.0, 'dress')]

Top 10 recommended new items for user 968500: [(27119.0, 'jumpsuit'), (11507.0, 'dress'), (11966.0, 'dress'), (27357.0, 'blazer'), (4816.0, 'sheath'), (544.0, 'sheath'), (17879.0, 'dress'), (528.0, 'dress'), (10883.0, 'sheath'), (2171.0, 'dress')]
-------------------------------------------------------------
Random 10 rented items for user 806600: [(14306.0, 'dress')]

Top 10 recommended new items for user 806600: [(27119.0, 'jumpsuit'), (11507.0, 'dress'), (11966.0, 'dress'), (27357.0, 'blazer'), (4816.0, 'sheath'), (544.0, 'sheath'), (17879.0, 'dress'), (528.0, 'dress'), (10883.0, 'sheath'), (2171.0, 'dress')]
-------------------------------------------------------------
Random 10 rented items for user 84770: [(14306.0, 'dress'), (21954.0, 'shift'), (10377.0, 'dress')]

Top 10 recommended new items for user 84770: [(27119.0, 'jumpsuit'), (11507.0, 'dress'), (11966.0, 'dress'), (27357.0, 'blazer'), (481

Top 10 recommended new items for user 791274: [(27119.0, 'jumpsuit'), (11507.0, 'dress'), (11966.0, 'dress'), (27357.0, 'blazer'), (4816.0, 'sheath'), (544.0, 'sheath'), (17879.0, 'dress'), (528.0, 'dress'), (10883.0, 'sheath'), (2171.0, 'dress')]
-------------------------------------------------------------
Random 10 rented items for user 822475: [(70.0, 'dress'), (7748.0, 'sheath'), (11965.0, 'dress'), (1592.0, 'dress'), (27635.0, 'suit'), (23303.0, 'sheath'), (21013.0, 'sheath')]

Top 10 recommended new items for user 822475: [(27119.0, 'jumpsuit'), (11507.0, 'dress'), (11966.0, 'dress'), (27357.0, 'blazer'), (4816.0, 'sheath'), (544.0, 'sheath'), (17879.0, 'dress'), (528.0, 'dress'), (10883.0, 'sheath'), (2171.0, 'dress')]
-------------------------------------------------------------
Random 10 rented items for user 402094: [(6133.0, 'dress'), (28929.0, 'jumpsuit'), (1714.0, 'dress'), (1592.0, 'dress'), (1388.0, 'dress'), (1405.0, 'dress'), (2534.0, 'dress')]

Top 10 recommended new

Random 10 rented items for user 134802: [(21843.0, 'dress'), (13679.0, 'dress'), (22933.0, 'dress'), (7683.0, 'dress')]

Top 10 recommended new items for user 134802: [(27119.0, 'jumpsuit'), (11507.0, 'dress'), (11966.0, 'dress'), (27357.0, 'blazer'), (4816.0, 'sheath'), (544.0, 'sheath'), (17879.0, 'dress'), (528.0, 'dress'), (10883.0, 'sheath'), (2171.0, 'dress')]
-------------------------------------------------------------
Random 10 rented items for user 597738: [(22933.0, 'dress')]

Top 10 recommended new items for user 597738: [(27119.0, 'jumpsuit'), (11507.0, 'dress'), (11966.0, 'dress'), (27357.0, 'blazer'), (4816.0, 'sheath'), (544.0, 'sheath'), (17879.0, 'dress'), (528.0, 'dress'), (10883.0, 'sheath'), (2171.0, 'dress')]
-------------------------------------------------------------
Random 10 rented items for user 21347: [(22933.0, 'dress'), (21847.0, 'dress')]

Top 10 recommended new items for user 21347: [(27119.0, 'jumpsuit'), (11507.0, 'dress'), (11966.0, 'dress'), (27357.

Top 10 recommended new items for user 411872: [(27119.0, 'jumpsuit'), (11507.0, 'dress'), (11966.0, 'dress'), (27357.0, 'blazer'), (4816.0, 'sheath'), (544.0, 'sheath'), (17879.0, 'dress'), (528.0, 'dress'), (10883.0, 'sheath'), (2171.0, 'dress')]
-------------------------------------------------------------
Random 10 rented items for user 517738: [(5781.0, 'dress')]

Top 10 recommended new items for user 517738: [(27119.0, 'jumpsuit'), (11507.0, 'dress'), (11966.0, 'dress'), (27357.0, 'blazer'), (4816.0, 'sheath'), (544.0, 'sheath'), (17879.0, 'dress'), (528.0, 'dress'), (10883.0, 'sheath'), (2171.0, 'dress')]
-------------------------------------------------------------
Random 10 rented items for user 51748: [(5781.0, 'dress')]

Top 10 recommended new items for user 51748: [(27119.0, 'jumpsuit'), (11507.0, 'dress'), (11966.0, 'dress'), (27357.0, 'blazer'), (4816.0, 'sheath'), (544.0, 'sheath'), (17879.0, 'dress'), (528.0, 'dress'), (10883.0, 'sheath'), (2171.0, 'dress')]
------------

...

Top 10 recommended new items for user 804772: [(27119.0, 'jumpsuit'), (11507.0, 'dress'), (11966.0, 'dress'), (27357.0, 'blazer'), (4816.0, 'sheath'), (544.0, 'sheath'), (17879.0, 'dress'), (528.0, 'dress'), (10883.0, 'sheath'), (2171.0, 'dress')]
-------------------------------------------------------------
Random 10 rented items for user 591660: [(590.0, 'gown'), (820.0, 'gown')]

Top 10 recommended new items for user 591660: [(27119.0, 'jumpsuit'), (11507.0, 'dress'), (11966.0, 'dress'), (27357.0, 'blazer'), (4816.0, 'sheath'), (544.0, 'sheath'), (17879.0, 'dress'), (528.0, 'dress'), (10883.0, 'sheath'), (2171.0, 'dress')]
-------------------------------------------------------------
Random 10 rented items for user 71691: [(4608.0, 'gown')]

Top 10 recommended new items for user 71691: [(27119.0, 'jumpsuit'), (11507.0, 'dress'), (11966.0, 'dress'), (27357.0, 'blazer'), (4816.0, 'sheath'), (544.0, 'sheath'), (17879.0, 'dress'), (528.0, 'dress'), (10883.0, 'sheath'), (2171.0, 'dress')

Random 10 rented items for user 955875: [(6732.0, 'dress')]

Top 10 recommended new items for user 955875: [(27119.0, 'jumpsuit'), (11507.0, 'dress'), (11966.0, 'dress'), (27357.0, 'blazer'), (4816.0, 'sheath'), (544.0, 'sheath'), (17879.0, 'dress'), (528.0, 'dress'), (10883.0, 'sheath'), (2171.0, 'dress')]
-------------------------------------------------------------
Random 10 rented items for user 521950: [(6732.0, 'dress')]

Top 10 recommended new items for user 521950: [(27119.0, 'jumpsuit'), (11507.0, 'dress'), (11966.0, 'dress'), (27357.0, 'blazer'), (4816.0, 'sheath'), (544.0, 'sheath'), (17879.0, 'dress'), (528.0, 'dress'), (10883.0, 'sheath'), (2171.0, 'dress')]
-------------------------------------------------------------
Random 10 rented items for user 67610: [(5093.0, 'dress')]

Top 10 recommended new items for user 67610: [(27119.0, 'jumpsuit'), (11507.0, 'dress'), (11966.0, 'dress'), (27357.0, 'blazer'), (4816.0, 'sheath'), (544.0, 'sheath'), (17879.0, 'dress'), (528.0, '

Top 10 recommended new items for user 957763: [(27119.0, 'jumpsuit'), (11507.0, 'dress'), (11966.0, 'dress'), (27357.0, 'blazer'), (4816.0, 'sheath'), (544.0, 'sheath'), (17879.0, 'dress'), (528.0, 'dress'), (10883.0, 'sheath'), (2171.0, 'dress')]
-------------------------------------------------------------
Random 10 rented items for user 101474: [(23056.0, 'dress')]

Top 10 recommended new items for user 101474: [(27119.0, 'jumpsuit'), (11507.0, 'dress'), (11966.0, 'dress'), (27357.0, 'blazer'), (4816.0, 'sheath'), (544.0, 'sheath'), (17879.0, 'dress'), (528.0, 'dress'), (10883.0, 'sheath'), (2171.0, 'dress')]
-------------------------------------------------------------
Random 10 rented items for user 369440: [(19597.0, 'gown')]

Top 10 recommended new items for user 369440: [(27119.0, 'jumpsuit'), (11507.0, 'dress'), (11966.0, 'dress'), (27357.0, 'blazer'), (4816.0, 'sheath'), (544.0, 'sheath'), (17879.0, 'dress'), (528.0, 'dress'), (10883.0, 'sheath'), (2171.0, 'dress')]
---------

Top 10 recommended new items for user 946684: [(27119.0, 'jumpsuit'), (11507.0, 'dress'), (11966.0, 'dress'), (27357.0, 'blazer'), (4816.0, 'sheath'), (544.0, 'sheath'), (17879.0, 'dress'), (528.0, 'dress'), (10883.0, 'sheath'), (2171.0, 'dress')]
-------------------------------------------------------------
Random 10 rented items for user 506643: [(2464.0, 'dress'), (1082.0, 'dress')]

Top 10 recommended new items for user 506643: [(27119.0, 'jumpsuit'), (11507.0, 'dress'), (11966.0, 'dress'), (27357.0, 'blazer'), (4816.0, 'sheath'), (544.0, 'sheath'), (17879.0, 'dress'), (528.0, 'dress'), (10883.0, 'sheath'), (2171.0, 'dress')]
-------------------------------------------------------------
Random 10 rented items for user 111842: [(927.0, 'gown')]

Top 10 recommended new items for user 111842: [(27119.0, 'jumpsuit'), (11507.0, 'dress'), (11966.0, 'dress'), (27357.0, 'blazer'), (4816.0, 'sheath'), (544.0, 'sheath'), (17879.0, 'dress'), (528.0, 'dress'), (10883.0, 'sheath'), (2171.0, 'dr

Top 10 recommended new items for user 500817: [(27119.0, 'jumpsuit'), (11507.0, 'dress'), (11966.0, 'dress'), (27357.0, 'blazer'), (4816.0, 'sheath'), (544.0, 'sheath'), (17879.0, 'dress'), (528.0, 'dress'), (10883.0, 'sheath'), (2171.0, 'dress')]
-------------------------------------------------------------
Random 10 rented items for user 456267: [(752.0, 'dress')]

Top 10 recommended new items for user 456267: [(27119.0, 'jumpsuit'), (11507.0, 'dress'), (11966.0, 'dress'), (27357.0, 'blazer'), (4816.0, 'sheath'), (544.0, 'sheath'), (17879.0, 'dress'), (528.0, 'dress'), (10883.0, 'sheath'), (2171.0, 'dress')]
-------------------------------------------------------------
Random 10 rented items for user 517328: [(17278.0, 'dress')]

Top 10 recommended new items for user 517328: [(27119.0, 'jumpsuit'), (11507.0, 'dress'), (11966.0, 'dress'), (27357.0, 'blazer'), (4816.0, 'sheath'), (544.0, 'sheath'), (17879.0, 'dress'), (528.0, 'dress'), (10883.0, 'sheath'), (2171.0, 'dress')]
----------

In [17]:
## compare results for BaselineOnly

df_exist = pd.DataFrame.from_dict(BsO_exist_dict,orient='index',\
                                  columns=['rent1','rent2','rent3',\
                                           'rent4','rent5','rent6',\
                                           'rent7','rent8','rent9',\
                                           'rent10']).reset_index()

df_rec = pd.DataFrame.from_dict(BsO_recc_dict, orient='index',\
                                columns=['rec1','rec2','rec3','rec4',\
                                         'rec5','rec6','rec7','rec8',\
                                         'rec9','rec10']).reset_index()

df_compare = pd.concat([df_exist, df_rec], axis=1).\
rename(columns={'index':'item ID'})

df_compare

,item ID,rent1,rent2,rent3,rent4,rent5,rent6,rent7,rent8,rent9,rent10,item ID,rec1,rec2,rec3,rec4,rec5,rec6,rec7,rec8,rec9,rec10
0,610914,"(751.0, dress)",None,None,None,None,None,None,None,None,None,610914,"(27119.0, jumpsuit)","(11507.0, dress)","(11966.0, dress)","(27357.0, blazer)","(4816.0, sheath)","(544.0, sheath)","(17879.0, dress)","(528.0, dress)","(10883.0, sheath)","(2171.0, dress)"
1,53519,"(751.0, dress)","(8003.0, dress)",None,None,None,None,None,None,None,None,53519,"(27119.0, jumpsuit)","(11507.0, dress)","(11966.0, dress)","(27357.0, blazer)","(4816.0, sheath)","(544.0, sheath)","(17879.0, dress)","(528.0, dress)","(10883.0, sheath)","(2171.0, dress)"
2,470639,"(414.0, gown)","(528.0, dress)","(22588.0, dress)","(751.0, dress)","(878.0, dress)",None,None,None,None,None,470639,"(27119.0, jumpsuit)","(11507.0, dress)","(11966.0, dress)","(27357.0, blazer)","(4816.0, sheath)","(544.0, sheath)","(17879.0, dress)","(10883.0, sheath)","(2171.0, dress)","(14597.0, dress)"
3,276186,"(804.0, dress)",None,None,None,None,None,None,None,None,None,276186,"(27119.0, jumpsuit)","(11507.0, dress)","(11966.0, dress)","(27357.0, blazer)","(4816.0, sheath)","(544.0, sheath)","(17879.0, dress)","(528.0, dress)","(10883.0, sheath)","(2171.0, dress)"
4,748397,"(17879.0, dress)","(800.0, sheath)","(5093.0, dress)","(804.0, dress)","(26856.0, jumpsuit)",None,None,None,None,None,748397,"(27119.0, jumpsuit)","(11507.0, dress)","(11966.0, dress)","(27357.0, blazer)","(4816.0, sheath)","(544.0, sheath)","(528.0, dress)","(10883.0, sheath)","(2171.0, dress)","(14597.0, dress)"
5,856829,"(9499.0, dress)","(1276.0, mini)","(804.0, dress)","(545.0, sheath)",None,None,None,None,None,None,856829,"(27119.0, jumpsuit)","(11507.0, dress)","(11966.0, dress)","(27357.0, blazer)","(4816.0, sheath)","(544.0, sheath)","(17879.0, dress)","(528.0, dress)","(10883.0, sheath)","(2171.0, dress)"
6,990544,"(804.0, dress)",None,None,None,None,None,None,None,None,None,990544,"(27119.0, jumpsuit)","(11507.0, dress)","(11966.0, dress)","(27357.0, blazer)","(4816.0, sheath)","(544.0, sheath)","(17879.0, dress)","(528.0, dress)","(10883.0, sheath)","(2171.0, dress)"
7,214108,"(8157.0, dress)",None,None,None,None,None,None,None,None,None,214108,"(27119.0, jumpsuit)","(11507.0, dress)","(11966.0, dress)","(27357.0, blazer)","(4816.0, sheath)","(544.0, sheath)","(17879.0, dress)","(528.0, dress)","(10883.0, sheath)","(2171.0, dress)"
8,503301,"(524.0, dress)","(1387.0, dress)","(8157.0, dress)","(1450.0, dress)",None,None,None,None,None,None,503301,"(27119.0, jumpsuit)","(11507.0, dress)","(11966.0, dress)","(27357.0, blazer)","(4816.0, sheath)","(544.0, sheath)","(17879.0, dress)","(528.0, dress)","(10883.0, sheath)","(2171.0, dress)"
9,574571,"(16499.0, dress)","(1005.0, mini)","(8157.0, dress)","(1602.0, dress)",None,None,None,None,None,None,574571,"(27119.0, jumpsuit)","(11507.0, dress)","(11966.0, dress)","(27357.0, blazer)","(4816.0, sheath)","(544.0, sheath)","(17879.0, dress)","(528.0, dress)","(10883.0, sheath)","(2171.0, dress)"


<div class='alert alert-danger'>
**Other metrics to evaluate top N recommendations**:

[Source 1](https://gab41.lab41.org/recommender-systems-its-not-all-about-the-accuracy-562c7dceeaff) ; 
[Source 2](https://towardsdatascience.com/evaluation-metrics-for-recommender-systems-df56c6611093) ; 
[rec metrics Python library (from source 2)](https://github.com/statisticianinstilettos/recmetrics)

<div class='alert alert-warning'>
**Coverage**:
- <font color=red>what % of user-item space can be recommended?</font>
- **User coverage**: measures the percentage of users with at least one "good" recommendation (rating above a certain threshold). A higher value is better. (numUsers = trainset.n_users)
- **Item coverage**: measures the percentage of recommended items (with a rating above a certain threshold). A higher value is better. (numItems = trainset.n_items)

In [18]:
## % of items in train set that model is able to recommend on a test set

def Coverage(predicted, length):
    """
    Computes the coverage for a list of recommendations
    Parameters:
    ----------
    predicted : a dictionary of lists (of item-rating pairs)
    
    length: integer
        The number of unique items in the whole training set
    
    Returns:
    ----------
    The coverage of the recommendations as a percent 
    rounded to 2 decimal places.
    """
    predicted_items = [iid for uid, user_ratings in predicted.items()\
                       for (iid, _) in user_ratings]
    unique_pred_items = len(set(predicted_items))
    coverage = round(unique_pred_items/(length*1.0)*100,2)/100
    return coverage

## for the top N recommendations, the train set is the whole dataset

print('Item coverage using BaselineOnly: {:.2%}'.\
      format(Coverage(topNPredicted_BsO,len(df))))

# print('Item coverage using SVDpp: {:.2%}'.\
#       format(Coverage(topNPredicted_SVDpp,len(df))))

# print('Item coverage using SVD: {:.2%}'.\
#       format(Coverage(topNPredicted_SVD,len(df))))

# print('Item coverage using KNNWithZScore: {:.2%}'.\
#       format(Coverage(topNPredicted_KNNZ,len(df))))

Item coverage using BaselineOnly: 0.64%


In [19]:
def UserCoverage(topNPredicted, numUsers, min_rating=4.0):
    hits = 0
    for user in topNPredicted.keys():
        hit = False
        for user_ratings in topNPredicted[user]:
            if (user_ratings[1] >= min_rating):
                hit = True
                break
        if (hit):
            hits += 1

    return hits / numUsers

print('User coverage using BaselineOnly: {:.2%}'.\
      format(UserCoverage(topNPredicted_BsO, len(df))))

# print('User coverage using SVDpp: {:.2%}'.\
#       format(UserCoverage(topNPredicted_SVDpp, len(df))))

# print('User coverage using SVD: {:.2%}'.\
#       format(UserCoverage(topNPredicted_SVD, len(df))))

# print('User coverage using KNNWithZScore: {:.2%}'.\
#       format(UserCoverage(topNPredicted_KNNZ, len(df))))

User coverage using BaselineOnly: 40.29%


<div class='alert alert-warning'>
**Novelty**:
- <font color=red>how surprising are the recommendations?</font>
- measures how many unknown recommended items are to a user (serendipity vs popularity)
- <font color=red>popular items: must have the best rating of 5</font>
- higher novelty value = less popular items being recommended

In [20]:
df_cleaned = pd.read_csv('renttherunway_cleaned_new.csv')
print(df_cleaned.shape)

## popular items must have a rating of 5 using overall cleaned dataset

popular = df_cleaned[df_cleaned['rating']==5]['item_id_new'].value_counts()

## assign rank based on item count
ranking_pop = pd.DataFrame(popular).rename(columns={'item_id_new':\
                                                    'item_count'})
ranking_pop['rank'] = ranking_pop['item_count'].rank(method='dense',\
                                                     ascending=False)
ranking_pop.head()

(190004, 26)


,item_count,rank
168.0,245,1.0
1389.0,223,2.0
66.0,205,3.0
169.0,200,4.0
546.0,195,5.0


In [21]:
def Novelty(topNPredicted):
    n = 0
    total = 0
    for user in topNPredicted.keys():
        for rating in topNPredicted[user]:
            itemID = rating[0]
            rank = ranking_pop.loc[itemID,'rank']
#             print('Item ID: {0} is ranked {1}'.format(itemID,rank))
#             if rank > 259:
#                 print('error')
            total += rank
            n += 1
    return round(total / n, 2)

print('Novelty score using BaselineOnly: ', Novelty(topNPredicted_BsO))

# print('Novelty score using SVDpp: ',\
#        Novelty(topNPredicted_SVDpp))

# print('Novelty score using SVD: ', Novelty(topNPredicted_SVD))

# print('Novelty score using KNNWithZScore: ', Novelty(topNPredicted_KNNZ))

Novelty score using BaselineOnly:  71.12


<div class='alert alert-warning'>
#### compare algorithms using KFold and whole data set
- using metrics: `Precision@k` and `Recall@k` functions from Surprise library
<img src="./screenshots/precision_recall.png" width=300 />

- an item is <font color=blue>considered relevant</font> if its <font color=red>true rating $r_{ui}$</font> is <font color=red>greater than a given threshold</font>.
- an item is <font color=blue>considered recommended</font> if its <font color=red>estimated rating $\hat r_{ui}$ is greater than the threshold</font>, and if it is <font color=red>among the k highest estimated ratings</font>.

In [22]:
## use precision@k and recall@k functions from Surprise library

def precision_recall_at_k(predictions, k=10, threshold=5.0):
    '''Return precision and recall at k metrics for each user.'''

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r>=threshold) and (est>=threshold))
                              for (est, true_r) in user_ratings[:k])
        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1

        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1

    return precisions, recalls

In [23]:
# calculate average precision and recall for BaselineOnly

kf = KFold(n_splits=5)

bsl_options = {'method':'als', 'n_epochs': 5, 'reg_u':12, 'reg_i': 5}
algo = BaselineOnly(bsl_options=bsl_options)

pmetrics = {}
rmetrics = {}
a=1

for trainset, testset in kf.split(data1):
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=5,\
                                                threshold=5.0)

    # Precision and recall can then be averaged over all users
    pmetrics[a] = (sum(prec for prec in precisions.values())\
                    / len(precisions))
    rmetrics[a] = (sum(rec for rec in recalls.values()) / len(recalls))
    a +=1

print('average precision for BaselineOnly: ',\
      float(sum(pmetrics.values()))/len(pmetrics))

print('average recall for BaselineOnly: ',\
      float(sum(rmetrics.values()))/len(rmetrics))

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
average precision for BaselineOnly:  1.0
average recall for BaselineOnly:  0.28238508202640167
